## **Data Cleaning**

Data Cleaning was done to tailor the data to our research question. 

Data cleaning was done to create a dataset from the raw data for analysis. After opening the original public_data tsv file, we observed that several variables were not available publicly and were only available in the restricted dataset which we did not have access to. These variables include NAME, URL_NAME, and PROJECT_PAGE_LOCATION_COUNTRY. In addition, there were some rows that had missing values, particularly in the PROJECT_PAGE_LOCATION_STATE, and PROJECT_PAGE_LOCATION_COUNTY columns which may indicate that these are user-inputs that are optional. These columns were dropped.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split 
import seaborn as sns
import duckdb
# import datetime

We imported the data and printed out the first 5 rows to view the columns provided by the dataset.

In [2]:
public_data = pd.read_csv('data/public_data.tsv', delimiter='\t')
public_data.head()

/var/folders/wp/nn2_qt4x0p96hfrk7plld7hw0000gn/T/ipykernel_7577/4156237419.py:1: DtypeWarning: Columns (9,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  public_data = pd.read_csv('data/public_data.tsv', delimiter='\t')


,CASEID,NAME,PID,CATEGORY,SUBCATEGORY,PROJECT_PAGE_LOCATION_NAME,PROJECT_PAGE_LOCATION_STATE,PROJECT_PAGE_LOCATION_COUNTY,PROJECT_PAGE_LOCATION_COUNTRY,UID,PROJECT_CURRENCY,GOAL_IN_ORIGINAL_CURRENCY,PLEDGED_IN_ORIGINAL_CURRENCY,GOAL_IN_USD,PLEDGED_IN_USD,BACKERS_COUNT,LAUNCHED_DATE,DEADLINE_DATE,STATE,URL_NAME
0,1,MASKED BY ICPSR,2137925650,11,301,London,England,Greater London,,1076478145,USD,"100,000,000",0,"$100,000,000",$0,0,8/11/2016,10/10/2016,canceled,MASKED BY ICPSR
1,2,MASKED BY ICPSR,1501531085,11,296,Los Angeles,CA,Los Angeles,,224946798,USD,"100,000,000",85,"$100,000,000",$85,4,12/19/2019,2/14/2020,canceled,MASKED BY ICPSR
2,3,MASKED BY ICPSR,953415668,16,51,Mexico,Baja California,Tijuana,,1772203542,MXN,"100,000,000",10,"$5,219,374",$1,1,3/1/2017,3/22/2017,failed,MASKED BY ICPSR
3,4,MASKED BY ICPSR,1371386304,18,18,Columbus,OH,Franklin,,1373465389,USD,"100,000,000",1,"$100,000,000",$1,1,6/4/2018,7/5/2018,canceled,MASKED BY ICPSR
4,5,MASKED BY ICPSR,1720842777,1,22,Toronto,ON,Toronto,,1455666383,CAD,"100,000,000",0,"$80,610,122",$0,0,5/1/2015,6/30/2015,failed,MASKED BY ICPSR


We checked for duplicates in the public_data.tsv dataset by selecting the rows that appeared more than once according to the unique PID. Since there are no rows that appeared more than once, we know that there is no duplicate PIDs in the dataset.

In [3]:
duckdb.sql("SELECT PID, COUNT(PID) FROM public_data GROUP BY PID HAVING COUNT(PID) > 1").df()

,PID,count(PID)


We dropped the following columns: NAME, PROJECT_PAGE_LOCATION_COUNTRY, and URL_NAME since they contained restricted information that wasn't available to the public. We also dropped the PROJECT_PAGE_LOCATION_STATE and PROJECT_PAGE_LOCATION_COUNTY since they were prone to user-input error and it is not plausible to clean all 600,000+ entries manually.

We also dropped the UID, and the rows that are not successful or failed (canceled or suspended). By definition, a canceled project is one that the project leader has stopped voluntarily and a suspended project is one that was taken down by KickStarter moderators for violating their terms of service or other policies. We chose to drop the canceled and suspended projects because we do not have further information regarding why they were canceled or suspended, since we do not have access to the project's name or url in the public dataset available to us. We could not make use of canceled or suspended projects in our research question either, since cancellations and suspensions of a campaign are subject to individual's decisions and frequently changing Kickstarter terms and conditions/policies which may vary from individual to individual/year to year.

In [13]:
public_data_df = pd.DataFrame(public_data)
clean_data = public_data_df.drop(columns = ["CASEID", "PID","NAME", "SUBCATEGORY", 
"PROJECT_PAGE_LOCATION_COUNTRY", "URL_NAME", "UID", 
"PROJECT_PAGE_LOCATION_STATE", "PROJECT_PAGE_LOCATION_COUNTY"])

clean_data = duckdb.sql("SELECT * FROM clean_data WHERE STATE = 'successful' OR STATE = 'failed'").df()
print(clean_data.head())

   CATEGORY PROJECT_PAGE_LOCATION_NAME PROJECT_CURRENCY  \
0        12                      Osaka              JPY   
1        12                      Tokyo              JPY   
2        12                   Ishikawa              JPY   
3        12                  Chiba-shi              JPY   
4        16                      Tokyo              JPY   

  GOAL_IN_ORIGINAL_CURRENCY PLEDGED_IN_ORIGINAL_CURRENCY GOAL_IN_USD  \
0                68,000,000                   81,030,744    $623,968   
1                53,808,516                  481,621,841    $510,735   
2                45,000,000                   98,863,825    $414,327   
3                16,665,000                   23,606,140    $148,285   
4                15,000,000                   17,406,300    $133,229   

  PLEDGED_IN_USD BACKERS_COUNT LAUNCHED_DATE DEADLINE_DATE       STATE  
0       $743,538         12613     3/26/2018      5/5/2018  successful  
1     $4,571,420         46307     7/27/2020     8/29/2020  succes

We also changed all the numerical string values in the dataframe into integers for us to work with. 

In [14]:
clean_data["GOAL_IN_ORIGINAL_CURRENCY"] = clean_data["GOAL_IN_ORIGINAL_CURRENCY"].replace({',': ''}, 
                                                                regex=True).astype(int)
clean_data["PLEDGED_IN_ORIGINAL_CURRENCY"] = clean_data["PLEDGED_IN_ORIGINAL_CURRENCY"].replace({',': ''},
                                                                     regex=True).astype(int)
clean_data["GOAL_IN_USD"] = clean_data["GOAL_IN_USD"].replace({'[\$]': '',',': ''}, 
                                                    regex=True).astype(int)
clean_data["PLEDGED_IN_USD"] = clean_data["PLEDGED_IN_USD"].replace({'[\$]': '',',': ''}, 
                                                            regex=True).astype(int)
print(clean_data.head())

<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:7: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:7: SyntaxWarning: invalid escape sequence '\$'
/var/folders/wp/nn2_qt4x0p96hfrk7plld7hw0000gn/T/ipykernel_7577/2567195272.py:5: SyntaxWarning: invalid escape sequence '\$'
  clean_data["GOAL_IN_USD"] = clean_data["GOAL_IN_USD"].replace({'[\$]': '',',': ''},
/var/folders/wp/nn2_qt4x0p96hfrk7plld7hw0000gn/T/ipykernel_7577/2567195272.py:7: SyntaxWarning: invalid escape sequence '\$'
  clean_data["PLEDGED_IN_USD"] = clean_data["PLEDGED_IN_USD"].replace({'[\$]': '',',': ''},


   CATEGORY PROJECT_PAGE_LOCATION_NAME PROJECT_CURRENCY  \
0        12                      Osaka              JPY   
1        12                      Tokyo              JPY   
2        12                   Ishikawa              JPY   
3        12                  Chiba-shi              JPY   
4        16                      Tokyo              JPY   

   GOAL_IN_ORIGINAL_CURRENCY  PLEDGED_IN_ORIGINAL_CURRENCY  GOAL_IN_USD  \
0                   68000000                      81030744       623968   
1                   53808516                     481621841       510735   
2                   45000000                      98863825       414327   
3                   16665000                      23606140       148285   
4                   15000000                      17406300       133229   

   PLEDGED_IN_USD BACKERS_COUNT LAUNCHED_DATE DEADLINE_DATE       STATE  
0          743538         12613     3/26/2018      5/5/2018  successful  
1         4571420         46307     7/27/2020 

We also checked for NaaN values by checking if there were any rows where the BACKERS_COUNT is null and if it was an empty string. We didn't realize until we started creating scatterplots that there were other inputs like Strings containing a single space that were still lying in our data. 

We also checked for oddities and outliers. For instance, we found rows where the BACKERS_COUNT variable was Null so we cross examined the identified rows' PLEDGED_IN_ORIGINAL_CURRENCY columns to see if the value was 0 or not. If it was 0, then we replaced the NaaN values with 0, assuming there were no backers and thus no money pledged. 

In [6]:
backers_count_null = duckdb.sql("SELECT * FROM clean_data WHERE BACKERS_COUNT IS NULL ").df()
print("The number of rows where BACKERS_COUNT is Null: " + str(len(backers_count_null)))

backers_count_null = duckdb.sql("SELECT * FROM clean_data WHERE BACKERS_COUNT = ''  ").df()
print("The number of rows where BACKERS_COUNT is an Empty String: " + str(len(backers_count_null)))

backers_count_null = duckdb.sql("SELECT * FROM clean_data WHERE BACKERS_COUNT = ' ' ").df()
print("The number of rows where BACKERS_COUNT is a String containing a space: " + str(len(backers_count_null)))

clean_data = duckdb.sql("SELECT * FROM clean_data WHERE NOT BACKERS_COUNT = ' '").df()
print("Cleaned the rows that contained a String with a single space")

backers_count_null = duckdb.sql("SELECT * FROM clean_data WHERE BACKERS_COUNT = ' ' ").df()
print("The number of rows where BACKERS_COUNT is a String containing a space: " + str(len(backers_count_null)))
clean_data["BACKERS_COUNT"] = clean_data["BACKERS_COUNT"].astype(int)

The number of rows where BACKERS_COUNT is Null: 0
The number of rows where BACKERS_COUNT is an Empty String: 0
The number of rows where BACKERS_COUNT is a String containing a space: 56
Cleaned the rows that contained a String with a single space
The number of rows where BACKERS_COUNT is a String containing a space: 0


We converted the LAUNCHED_DATE and DEADLINE_DATE to datetime objects so we could do math with the two dates to get the length of each campaign.

We then calculated the difference between the DEADLINE_DATE and LAUNCHED_DATE to get the duration in days that each project fundraiser ran for and added it as a new column to the clean_data.

In [15]:
clean_data['LAUNCHED_DATE'] = pd.to_datetime(clean_data['LAUNCHED_DATE'], format = 'mixed')
clean_data['DEADLINE_DATE'] = pd.to_datetime(clean_data['DEADLINE_DATE'], format = 'mixed')

#new column for length of campaign
clean_data['duration_in_days'] = (clean_data['DEADLINE_DATE'] 
                                      - clean_data['LAUNCHED_DATE']).dt.days
print(clean_data.head())

   CATEGORY PROJECT_PAGE_LOCATION_NAME PROJECT_CURRENCY  \
0        12                      Osaka              JPY   
1        12                      Tokyo              JPY   
2        12                   Ishikawa              JPY   
3        12                  Chiba-shi              JPY   
4        16                      Tokyo              JPY   

   GOAL_IN_ORIGINAL_CURRENCY  PLEDGED_IN_ORIGINAL_CURRENCY  GOAL_IN_USD  \
0                   68000000                      81030744       623968   
1                   53808516                     481621841       510735   
2                   45000000                      98863825       414327   
3                   16665000                      23606140       148285   
4                   15000000                      17406300       133229   

   PLEDGED_IN_USD BACKERS_COUNT LAUNCHED_DATE DEADLINE_DATE       STATE  \
0          743538         12613    2018-03-26    2018-05-05  successful   
1         4571420         46307    2020-07-2

We changed the LAUNCHED_DATE and DEADLINE_DATE to only the corresponding year as we intend to analyzing the years that the campaigns were launched and finished in. 

The DEADLINE_DATE and LAUNCHED_DATE columns were then dropped.

In [16]:
clean_data['launched_year'] = clean_data['LAUNCHED_DATE'].dt.year
clean_data = clean_data.drop(columns = ["DEADLINE_DATE", "LAUNCHED_DATE"])
print(clean_data.head())

   CATEGORY PROJECT_PAGE_LOCATION_NAME PROJECT_CURRENCY  \
0        12                      Osaka              JPY   
1        12                      Tokyo              JPY   
2        12                   Ishikawa              JPY   
3        12                  Chiba-shi              JPY   
4        16                      Tokyo              JPY   

   GOAL_IN_ORIGINAL_CURRENCY  PLEDGED_IN_ORIGINAL_CURRENCY  GOAL_IN_USD  \
0                   68000000                      81030744       623968   
1                   53808516                     481621841       510735   
2                   45000000                      98863825       414327   
3                   16665000                      23606140       148285   
4                   15000000                      17406300       133229   

   PLEDGED_IN_USD BACKERS_COUNT       STATE  duration_in_days  launched_year  
0          743538         12613  successful                40           2018  
1         4571420         46307  suc

We then made the categorical variable, STATE, into a binary variable so we can run logit on it with other variables later to find possible predictors for the success and/or failure of a project.

In [9]:
clean_data["STATE"] = (clean_data["STATE"] =="successful").astype(int)
clean_data.head()

,CATEGORY,PROJECT_PAGE_LOCATION_NAME,PROJECT_CURRENCY,GOAL_IN_ORIGINAL_CURRENCY,PLEDGED_IN_ORIGINAL_CURRENCY,GOAL_IN_USD,PLEDGED_IN_USD,BACKERS_COUNT,STATE,duration_in_days,launched_year
0,12,Osaka,JPY,68000000,81030744,623968,743538,12613,1,40,2018
1,12,Tokyo,JPY,53808516,481621841,510735,4571420,46307,1,33,2020
2,12,Ishikawa,JPY,45000000,98863825,414327,910266,8051,1,8,2019
3,12,Chiba-shi,JPY,16665000,23606140,148285,210047,1705,1,30,2018
4,16,Tokyo,JPY,15000000,17406300,133229,154601,656,1,60,2017


### Renaming Columns

The fields in the column "CATEGORY" were renamed to make it clearer on what each category number represented based on information from the website the original raw data was taken from. For example, category 1 corresponds with "Art". 

In [10]:
print(clean_data["CATEGORY"].unique())
clean_data["CATEGORY"] = clean_data["CATEGORY"].astype(str)
clean_data["CATEGORY"] = clean_data["CATEGORY"].replace({"1":"Art", "3":"Comics", "6": "Dance", 
                                                         "7":"Design", "9":"Fashion", "10":"Food",
                                                         "11": "Film & Video", "12":"Games",
                                                         "13": "Journalism", "14":"Music", 
                                                         "15":"Photography", "16":"Technology",
                                                         "17":"Theater", "18": "Publishing", 
                                                         "26": "Crafts"})
print(clean_data["CATEGORY"].unique())

[12 16 11  7 14  3 26 10  1 18  9 13 17 15  6]
['Games' 'Technology' 'Film & Video' 'Design' 'Music' 'Comics' 'Crafts'
 'Food' 'Art' 'Publishing' 'Fashion' 'Journalism' 'Theater' 'Photography'
 'Dance']


The columns were renamed to be more concise and intuitive to understand.

In [ ]:
clean_data = clean_data.rename(columns = {
    "CATEGORY" : "category",
    "PROJECT_PAGE_LOCATION_NAME" : "project_location",
    "PROJECT_CURRENCY" : "original_currency",
    "GOAL_IN_ORIGINAL_CURRENCY" : "goal_original_currency",
    "PLEDGED_IN_ORIGINAL_CURRENCY" : "pledged_original_currency",
    "GOAL_IN_USD" : "goal_usd",
    "PLEDGED_IN_USD" : "pledged_usd",
    "BACKERS_COUNT" : "num_backers",
    "STATE" : "success"})

print(clean_data.head())

   category project_location original_currency  goal_original_currency  \
0        12            Osaka               JPY                68000000   
1        12            Tokyo               JPY                53808516   
2        12         Ishikawa               JPY                45000000   
3        12        Chiba-shi               JPY                16665000   
4        16            Tokyo               JPY                15000000   

   pledged_original_currency  goal_usd  pledged_usd num_backers     success  \
0                   81030744    623968       743538       12613  successful   
1                  481621841    510735      4571420       46307  successful   
2                   98863825    414327       910266        8051  successful   
3                   23606140    148285       210047        1705  successful   
4                   17406300    133229       154601         656  successful   

   duration_in_days  launched_year  
0                40           2018  
1     

### Exporting to CSV

In [12]:
clean_data = clean_data.to_csv("kickstarter_data.csv", index = False)